In [72]:
# Import all cars from opendbc

from opendbc.car import structs
from opendbc.car.values import PLATFORMS as TEST_PLATFORMS

# Example: add additional platforms/segments to test outside of commaCarSegments

EXTRA_SEGMENTS = {
  "81dd9e9fe256c397/0000001f--97c42cf98d",  # Volkswagen ID.4 test route
}

In [73]:
import random

from openpilot.tools.lib.logreader import LogReader
from openpilot.tools.lib.comma_car_segments import get_comma_car_segments_database


MAX_SEGS_PER_PLATFORM = 3  # Increase this to search more segments

database = get_comma_car_segments_database()
TEST_SEGMENTS = []

print(f"Searching {len(TEST_PLATFORMS)} platforms")

for platform in TEST_PLATFORMS:
  if platform not in database:
    print(f"No segments available for {platform}")
    continue
  
  all_segments = database[platform]
  NUM_SEGMENTS = min(len(all_segments), MAX_SEGS_PER_PLATFORM)
  TEST_SEGMENTS.extend(random.sample(all_segments, NUM_SEGMENTS))

TEST_SEGMENTS.extend(EXTRA_SEGMENTS)

print(f"Searching {len(TEST_SEGMENTS)} segments")

Searching 221 platforms
No segments available for DODGE_DURANGO
No segments available for FORD_RANGER_MK2
No segments available for HOLDEN_ASTRA
No segments available for CADILLAC_ATS
No segments available for CHEVROLET_MALIBU
No segments available for CADILLAC_XT4
No segments available for CHEVROLET_VOLT_2019
No segments available for CHEVROLET_TRAVERSE
No segments available for GMC_YUKON
No segments available for HONDA_ODYSSEY_CHN
No segments available for HYUNDAI_KONA_2022
No segments available for HYUNDAI_NEXO_1ST_GEN
No segments available for GENESIS_GV70_ELECTRIFIED_1ST_GEN
No segments available for GENESIS_G80_2ND_GEN_FL
No segments available for RIVIAN_R1_GEN1
No segments available for SUBARU_FORESTER_HYBRID
No segments available for TESLA_MODEL_3
No segments available for TESLA_MODEL_Y
No segments available for TOYOTA_RAV4_PRIME
No segments available for TOYOTA_SIENNA_4TH_GEN
No segments available for LEXUS_LC_TSS2
No segments available for VOLKSWAGEN_CADDY_MK3
No segments ava

In [74]:
from openpilot.tools.lib.logreader import LogReader
from tqdm.notebook import tqdm, tnrange

# Example search for CAN ignition messages
# Be careful when filtering by bus, account for external Red Panda or odd harness arrangements on Honda/HKG

BUSES_TO_SEARCH = [0, 1, 2]

MESSAGES_TO_FIND = {
  0x1F1: "GM CAN Ign",
  0x152: "Rivian CAN Ign",
  0x221: "Tesla 3/Y CAN Ign",
  0x9E:  "Mazda CAN Ign",
  0x3C0: "VW CAN Ign",
}

progress_bar = tnrange(len(TEST_SEGMENTS), desc="segments searched")

for segment in TEST_SEGMENTS:
  lr = LogReader(segment)
  CP = lr.first("carParams")
  if CP is None:
    progress_bar.update()
    continue

  can_packets = [msg for msg in lr if msg.which() == "can"]
  matched_messages = set()

  for packet in can_packets:
    for msg in packet.can:
      if msg.address in MESSAGES_TO_FIND and msg.src in BUSES_TO_SEARCH:
        # print(msg)
        matched_messages.add(msg.address)

  if len(matched_messages) > 0:
    message_names = [MESSAGES_TO_FIND[message] for message in matched_messages]
    print(f"Match found: {segment:<45} {CP.carFingerprint:<38} {message_names}")

  progress_bar.update()


segments searched:   0%|          | 0/578 [00:00<?, ?it/s]

Match found: 7f268b92fc672c4d/2023-11-02--20-54-34/133/s   CHRYSLER PACIFICA HYBRID 2018          ['VW CAN Ign']
Match found: daca9770319063f1/2023-12-12--14-28-25/5/s     CHRYSLER PACIFICA HYBRID 2018          ['VW CAN Ign']
Match found: 796aba999d046d7d/2023-11-04--15-43-47/10/s    CHRYSLER PACIFICA HYBRID 2018          ['VW CAN Ign']
Match found: cf10bbff2c1d2c6b/2023-10-29--17-05-05/5/s     CHRYSLER PACIFICA HYBRID 2019          ['VW CAN Ign']
Match found: 98d310eca1a221ba/2023-12-26--11-50-34/5/s     CHRYSLER PACIFICA HYBRID 2019          ['VW CAN Ign']
Match found: f800ed88a5f7fa19/2023-11-19--16-17-48/9/s     CHRYSLER PACIFICA HYBRID 2019          ['VW CAN Ign']
Match found: 8363f866866f25e7/2023-05-10--06-55-11/26/s    CHRYSLER PACIFICA 2018                 ['VW CAN Ign']
Match found: d5de6f3e9371e89e/2023-11-22--15-37-03/2/s     CHRYSLER PACIFICA 2018                 ['VW CAN Ign']
Match found: d5de6f3e9371e89e/2023-11-16--17-19-54/64/s    CHRYSLER PACIFICA 2018               